In [15]:
import pandas as pd
import numpy as np
import re
import os
from glob import glob
PATH = '../../Results'
EXT = "*.csv"

# final pipeline for all features in all results

In [16]:
nan = np.nan
def using_clump(a):
    return [a[s] for s in np.ma.clump_unmasked(np.ma.masked_where(a.astype(str)==str(np.nan),a))]
def centroids_zero_center(tracks_arr):
    for centroids_arr in tracks_arr:
        centroids_arr-=centroids_arr[0]
    return tracks_arr
numeric_const_pattern = '[-+]? (?: (?: \d* \. \d+ ) | (?: \d+ \.? ) )(?: [Ee] [+-]? \d+ ) ?'
rx = re.compile(numeric_const_pattern, re.VERBOSE)
def str_array_to_float(arr_of_arr_of_str):
    final_mat = []
    for arr_of_arr in arr_of_arr_of_str:
        float_ts = []
        for str in arr_of_arr:
            float_arr = [float(i) for i in rx.findall(str)]
            if(len(float_arr)>=2):
                float_ts.append(float_arr)
        if(len(float_ts)>=1):
            final_mat.append(float_ts)
    return np.array(final_mat,dtype=object)
def get_lens(tracks):
    return pd.Series([len(trk) for trk in tracks]).value_counts()
def get_feature_index(feature_type,features):
    for idx,fet in enumerate(features):
        if fet in feature_type:
            return idx
    return 0
def cut_feture_vecs_and_preprocess(tracks,feature_type,ts_len,cut_longer_ts=False):
    if(cut_longer_ts):
        track_cut = np.array([trk[:ts_len] for trk in tracks if len(trk)>=ts_len])
    else:
        tracks_cut = np.array([trk for trk in tracks if len(trk)==ts_len])
    if 'centroids' in feature_type:
        track_cut = centroids_zero_center(track_cut)
    return track_cut
def save(tracks_final,well_name):
    np.save('../npy_files/'+well_name+'.npy',tracks_final)

In [24]:
def from_results_folder_PATH_to_arrays(features=['centroids','morphologies','embeddings'],ts_len=10,cut_longer_ts=False,save=False,name_ext=""):
    all_tracks = []
    wells = []
    all_paths = [path for path, subdir, files in os.walk(PATH)]
    for path in all_paths:
        feature_vecs_cut = []
        all_files = [file for file in glob(os.path.join(path, EXT))]
        if(len(all_files)<1):
            continue
        for file in all_files:
            file_name = file.split('_')
            well_name = file_name[1]
            feature_type = file_name[-1]
            if(not any(fet in feature_type for fet in features)):
                continue
            df_str = pd.read_csv(file,index_col=[0])
            splitted = []
            for cell_id, series in df_str.iterrows():
                tracks = np.array(using_clump(np.array(series)),dtype=object)
                for tr in tracks:
                    splitted.append(tr)
            tracks_str = np.array(splitted,dtype=object) 
            #print("tracks_str shape: ",tracks_str.shape)
            tracks = str_array_to_float(tracks_str)
            tracks_cut = cut_feture_vecs_and_preprocess(tracks,feature_type,ts_len,cut_longer_ts)
            feature_vecs_cut.append(tracks_cut)
        feature_vecs_cut = np.dstack(feature_vecs_cut)
        if(len(feature_vecs_cut[0])>0):
            print(feature_vecs_cut.shape)
            all_tracks.append(feature_vecs_cut)
            wells.append(well_name)
    #return all_tracks,wells
    labels = []
    for well_name,tracks_vec in zip(wells,all_tracks):
        labels.append(np.repeat(well_name,len(tracks_vec)))
    return all_tracks
    results_tracks = np.vstack(all_tracks)
    results_labels = np.concatenate(labels)
    if(save):
        np.save('../npy_files/features'+name_ext+'.npy',results_tracks)
        np.save('../npy_files/labels'+name_ext+'.npy',results_labels)
    return results_tracks,results_labels

In [25]:
a = from_results_folder_PATH_to_arrays(features=['centroids','morphologies'],ts_len=25,cut_longer_ts=True,save=True,name_ext="_Asaf")

In [39]:
len(a[1][1])

25

In [52]:
type(a)

list

In [57]:
new_list =[]
for i,l in enumerate(a):
    for j, k in enumerate(l):
        if len(a[i]) == 25:
            print(i,j)
            new_list.append(a[i])


In [55]:
len(new_list)

0

In [21]:
x,y = from_results_folder_PATH_to_arrays(features=['centroids','morphologies'],ts_len=25,cut_longer_ts=True,save=True,name_ext="_Asaf")

../../Results
../../Results\Results_D2_1_start_3_crop_0_2022-08-29_19-03-32
../../Results\Results_D2_2_start_2_crop_0_2022-08-29_18-52-37
../../Results\Results_D2_3_start_2_crop_0_2022-08-29_18-45-16
../../Results\Results_D2_3_start_2_crop_3_2022-08-29_18-49-31
../../Results\Results_D2_3_start_5_crop_1_2022-08-29_18-46-42
../../Results\Results_D2_4_start_2_crop_0_2022-08-29_18-58-00
../../Results\Results_D3_1_start_5_crop_2_2022-08-29_19-07-07
../../Results\Results_D3_1_start_6_crop_1_2022-08-29_19-06-14
../../Results\Results_D3_1_start_6_crop_3_2022-08-29_19-09-12
../../Results\Results_D3_1_start_9_crop_0_2022-08-29_19-05-03
../../Results\Results_D3_2_start_4_crop_0_2022-08-29_19-10-51
../../Results\Results_D3_2_start_4_crop_2_2022-08-29_19-14-06
../../Results\Results_D3_2_start_6_crop_1_2022-08-29_19-13-07
../../Results\Results_D3_2_start_7_crop_3_2022-08-29_19-15-01
../../Results\Results_D3_3_start_2_crop_1_2022-08-29_19-19-26
../../Results\Results_D3_3_start_4_crop_0_2022-08-29_19-

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 25 and the array at index 19 has size 0

In [14]:
x.shape

(58, 25, 5)

## tests and single fetures

In [3]:
nan = np.nan
def using_clump(a):
    return [a[s] for s in np.ma.clump_unmasked(np.ma.masked_where(a.astype(str)==str(np.nan),a))]
def normalize_centroids_in_tracks(tracks_arr):
    for centroids_arr in tracks_arr:
        centroids_arr-=centroids_arr[0]
    return tracks_arr
def str_array_to_float(arr_of_arr_of_str):
    final_mat = []
    for arr_of_arr in arr_of_arr_of_str:
        float_ts = []
        for str in arr_of_arr:
            float_arr = [float(i) for i in re.findall("\d+\.\d+",str)]
            if(len(float_arr)>=2):
                float_ts.append(float_arr)
        if(len(float_ts)>=1):
            final_mat.append(float_ts)
    return np.array(final_mat)
def get_lens(tracks):
    return pd.Series([len(trk) for trk in tracks]).value_counts()
def cut_tracks_and_save(tracks,well_name,ts_len,cut_longer_ts=False,save=True):
    if(cut_longer_ts):
        tracks_final = np.array([trk[:ts_len] for trk in tracks if len(trk)>=ts_len])
    else:
        tracks_final = np.array([trk for trk in tracks if len(trk)==ts_len])
    track_final_norm=normalize_centroids_in_tracks(tracks_final)
    if(save):
        np.save('../npy_files/'+well_name+'.npy',track_final_norm)
    return track_final_norm

In [15]:
def from_csv_path_to_npy(csv_path,well_name,ts_len,cut_longer_ts=False):
    df_str = pd.read_csv(csv_path,index_col=[0])
    splitted = []
    id_well_index = []
    for cell_id, series in df_str.iterrows():
        tracks = np.array(using_clump(np.array(series)))
        for tr in tracks:
            splitted.append(tr)
    tracks_str = np.array(splitted) 
    print("tracks_str shape: ",tracks_str.shape)
    tracks = str_array_to_float(tracks_str)
    cut_tracks_and_save(tracks,well_name,10,False)

In [5]:
def from_results_folder_PATH_to_arrays():
    all_tracks = []
    wells = []
    all_csv_files = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]
    for file in all_csv_files:
        file_name = file.split('_')
        well_name = file_name[1]
        df_str = pd.read_csv(file,index_col=[0])
        splitted = []
        id_well_index = []
        for cell_id, series in df_str.iterrows():
            tracks = np.array(using_clump(np.array(series)))
            for tr in tracks:
                splitted.append(tr)
        tracks_str = np.array(splitted) 
        print("tracks_str shape: ",tracks_str.shape)
        tracks = str_array_to_float(tracks_str)
        all_tracks.append(tracks)
        wells.append(well_name)
    return all_tracks,wells

### tracks


In [5]:
PATH = '../Results'
EXT = "*centroids.csv"
all_tracks,wells = from_results_folder_PATH_to_arrays()

UnboundLocalError: local variable 'track_cut' referenced before assignment

In [107]:
all = []
labels = []
for well_name,tracks in zip(wells,all_tracks):
    tracks_norm_cut = cut_tracks_and_save(tracks,well_name,12,True)
    all.append(tracks_norm_cut)
    labels.append(np.repeat(well_name,len(tracks_norm_cut)))

results_tracks = np.vstack(all)
results_labels = np.concatenate(labels)


In [108]:
len(results_tracks)

3705

In [109]:
np.save('../npy_files/tracks.npy',results_tracks)
np.save('../npy_files/labels.npy',results_labels)

### morphologies

In [ ]:
PATH = '../Results'
EXT = "*morphologies.csv"
all_tracks,wells = from_results_folder_PATH_to_arrays()

In [8]:
all = []
labels = []
for well_name,tracks in zip(wells,all_tracks):
    tracks_norm_cut = cut_tracks_and_save(tracks,well_name,7,True)
    all.append(tracks_norm_cut)
    labels.append(np.repeat(well_name,len(tracks_norm_cut)))

results_tracks = np.vstack(all)
results_labels = np.concatenate(labels)

In [9]:
len(results_tracks)

8931

In [10]:
np.save('../npy_files/morph.npy',results_tracks) 
np.save('../npy_files/labels_morph.npy',results_labels)

#### file tests

In [12]:
PATH = '../Results_All'
EXT = "*.csv"
all_csv_files = [file
                for path, subdir, files in os.walk(PATH)
                for file in glob(os.path.join(path, EXT))]
                
all_paths = [path for path, subdir, files in os.walk(PATH)]
lens = []
for path in all_paths:
    all_files = [file for file in glob(os.path.join(path, EXT)) ]
    print(len(all_files)) 
   

0
2
1
0
2
2
1
3
3
1
1
1
2
3
1
2
0
1
2
0
0
1
0
0
0
3
1
0
0
1
1
0
0
2
0
1
1
3
2
2
0
0
2
2
1
2
0
2
2
1
1
3
0
2
3
1
1
1
2
0
3
0
0
2
0
2
0
2
3
2
0
0
3
2
2
3
1
1
2
1
3
1
0
3
2
2
1
3
3
1
3
1
1
2
1
2
1
3
0
3
2
1
1


NameError: name 'Results' is not defined

In [ ]:
for file in all_csv_files:
    file_name = file.split('_')
    print(file_name[-1])

In [ ]:
for file in all_csv_files:
    file_name = file.split('_')
    well_name = file_name[1]
    view_name = file_name[2]
    crop_name = file_name[4]
    print(f'well:{well_name} , view:{view_name}, crop:{crop_name}')

# old code

### path to a centroids.csv
comment out mac/windows file depending on os you are using

In [16]:
df_t = pd.read_csv('/Users/amosavni/university/DeepLearningWorkshop/deepcell_mod/DL-WORKSHOP/results/Results_D2_4_crop_0_start_2_2022-08-25_11-10-58/centroids.csv',index_col=[0])

# strings df to array of array of strings

In [ ]:
splitted = []
id_well_index = []
for cell_id, series in df_t.iterrows():
    tracks = np.array(using_clump(np.array(series)))
    for tr in tracks:
        splitted.append(tr)

In [ ]:
tracks_str = np.array(splitted) 
print(tracks_str.shape)

In [ ]:
tracks = str_array_to_float(tracks_str)

In [ ]:
print(get_lens(tracks))

In [16]:
tracks_final = np.array([trk for trk in tracks if len(trk)==10])

In [18]:
track_final_norm=normalize_centroids_in_tracks(tracks_final)

In [21]:
np.save('track_final_norm.npy',track_final_norm)